In [1]:
from keras.layers import Dense, Input
from keras.models import Model

Using TensorFlow backend.


In [2]:
# number of features in reduced dimension
encoding_dim = 32

# input layer
input_img = Input(shape=(784,))

# encoded layer
encoded = Dense(encoding_dim, activation='relu')(input_img)

# decoded layer
decoded = Dense(784, activation='sigmoid')(encoded)

# maps input image to the encoded image
encoder = Model(input_img, encoded)

# maps input image to its reconstructed version
autoencoder = Model(inputs=input_img, outputs=decoded)

W0929 20:19:32.108077 4648940992 deprecation_wrapper.py:119] From /Users/ryan/.conda/envs/official_xtract/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0929 20:19:32.119141 4648940992 deprecation_wrapper.py:119] From /Users/ryan/.conda/envs/official_xtract/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0929 20:19:32.121424 4648940992 deprecation_wrapper.py:119] From /Users/ryan/.conda/envs/official_xtract/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [0]:
# input layer for encoded image
encoded_input = Input(shape=(encoding_dim,))

# decoder layer
decoder_layer = autoencoder.layers[-1]

# decoder model
decoder = Model(inputs=encoded_input, outputs=decoder_layer(encoded_input))

autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')
autoencoder.summary()

Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 784)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 32)                25120     
_________________________________________________________________
dense_8 (Dense)              (None, 784)               25872     
Total params: 50,992
Trainable params: 50,992
Non-trainable params: 0
_________________________________________________________________


In [3]:
# mnist processing

from keras.datasets import mnist
import keras
import numpy as np
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

x_train = x_train.reshape(len(x_train), 784)
x_test = x_test.reshape(len(x_test), 784)
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

print(x_train[0])

[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         

In [0]:
autoencoder.fit(x_train, x_train,
               epochs=50,
               batch_size=256,
               shuffle=True,
               validation_data=(x_test, x_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/50
60000/60000 [==============================] - 4s 74us/step - loss: 0.3505 - val_loss: 0.2709
Epoch 2/50
60000/60000 [==============================] - 4s 71us/step - loss: 0.2633 - val_loss: 0.2518
Epoch 3/50
60000/60000 [==============================] - 4s 71us/step - loss: 0.2421 - val_loss: 0.2304
Epoch 4/50
60000/60000 [==============================] - 4s 70us/step - loss: 0.2227 - val_loss: 0.2124
Epoch 5/50
60000/60000 [==============================] - 4s 71us/step - loss: 0.2069 - val_loss: 0.1990
Epoch 6/50
60000/60000 [==============================] - 4s 70us/step - loss: 0.1954 - val_loss: 0.1894
Epoch 7/50
60000/60000 [==============================] - 4s 70us/step - loss: 0.1867 - val_loss: 0.1815
Epoch 8/50
60000/60000 [==============================] - 4s 70us/step - loss: 0.1796 - val_loss: 0.1749
Epoch 9/50
60000/60000 [==============================] - 4s 69us/step - loss: 0.1735 - val_loss: 0.1694
Epoch

In [0]:
encoded_imgs = encoder.predict(x_test)
decoded_imgs = decoder.predict(encoded_imgs)

0.09862608340929965


In [4]:
# use Matplotlib (don't ask)
import matplotlib.pyplot as plt

n = 10  # how many digits we will display
plt.figure(figsize=(20, 4))
for i in range(n):
    # display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

NameError: name 'decoded_imgs' is not defined

In [0]:
from keras import regularizers

encoding_dim = 32

input_img = Input(shape=(784,))
# add a Dense layer with a L1 activity regularizer
encoded = Dense(encoding_dim, activation='relu',
                activity_regularizer=regularizers.l1(10e-5))(input_img)
decoded = Dense(784, activation='sigmoid')(encoded)

autoencoder = Model(input_img, decoded)

autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')
autoencoder.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 784)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 32)                25120     
_________________________________________________________________
dense_6 (Dense)              (None, 784)               25872     
Total params: 50,992
Trainable params: 50,992
Non-trainable params: 0
_________________________________________________________________


In [0]:
autoencoder.fit(x_train, x_train,
               epochs=100,
               batch_size=256,
               shuffle=True,
               validation_data=(x_test, x_test))


Train on 60000 samples, validate on 10000 samples
Epoch 1/100
60000/60000 [==============================] - 5s 91us/step - loss: 0.6729 - val_loss: 0.6485
Epoch 2/100
60000/60000 [==============================] - 4s 74us/step - loss: 0.6284 - val_loss: 0.6090
Epoch 3/100
60000/60000 [==============================] - 5s 80us/step - loss: 0.5916 - val_loss: 0.5749
Epoch 4/100
60000/60000 [==============================] - 4s 73us/step - loss: 0.5598 - val_loss: 0.5454
Epoch 5/100
60000/60000 [==============================] - 4s 75us/step - loss: 0.5323 - val_loss: 0.5198
Epoch 6/100
60000/60000 [==============================] - 5s 83us/step - loss: 0.5084 - val_loss: 0.4975
Epoch 7/100
60000/60000 [==============================] - 5s 86us/step - loss: 0.4875 - val_loss: 0.4780
Epoch 8/100
60000/60000 [==============================] - 5s 77us/step - loss: 0.4692 - val_loss: 0.4609
Epoch 9/100
60000/60000 [==============================] - 5s 78us/step - loss: 0.4531 - val_loss: 0.

In [5]:
# uses a deeper network to encode images

input_img = Input((784,))
encoded = Dense(128, activation='relu')(input_img)
encoded = Dense(64, activation='relu')(encoded)
encoded = Dense(32, activation='relu')(encoded)

decoded = Dense(64, activation='relu')(encoded)
decoded = Dense(128, activation='relu')(decoded)
decoded = Dense(784, activation='sigmoid')(decoded)

autoencoder = Model(inputs=input_img, outputs=decoded)
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy', metrics=['accuracy'])
autoencoder.fit(x_train, x_train,
               epochs=10,
               batch_size = 256,
               shuffle=True,
               validation_data=(x_test, x_test))

encoder = Model(input_img, encoded)

W0929 20:19:51.508165 4648940992 deprecation_wrapper.py:119] From /Users/ryan/.conda/envs/official_xtract/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0929 20:19:51.517754 4648940992 deprecation_wrapper.py:119] From /Users/ryan/.conda/envs/official_xtract/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3376: The name tf.log is deprecated. Please use tf.math.log instead.

W0929 20:19:51.521856 4648940992 deprecation.py:323] From /Users/ryan/.conda/envs/official_xtract/lib/python3.7/site-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0929 20:19:51.697019 4648940992 deprecation_wrapper.py:119] From /Users/ryan/.conda/envs/official_xtract/lib/python3.

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 2s 41us/step - loss: 0.3507 - acc: 0.7439 - val_loss: 0.2650 - val_acc: 0.7866
Epoch 2/10
60000/60000 [==============================] - 2s 34us/step - loss: 0.2597 - acc: 0.7948 - val_loss: 0.2573 - val_acc: 0.7946
Epoch 3/10
60000/60000 [==============================] - 2s 35us/step - loss: 0.2524 - acc: 0.7983 - val_loss: 0.2483 - val_acc: 0.7928
Epoch 4/10
60000/60000 [==============================] - 2s 34us/step - loss: 0.2423 - acc: 0.7952 - val_loss: 0.2368 - val_acc: 0.7920
Epoch 5/10
60000/60000 [==============================] - 2s 35us/step - loss: 0.2310 - acc: 0.7945 - val_loss: 0.2243 - val_acc: 0.7903
Epoch 6/10
60000/60000 [==============================] - 2s 35us/step - loss: 0.2174 - acc: 0.7949 - val_loss: 0.2093 - val_acc: 0.7968
Epoch 7/10
60000/60000 [==============================] - 2s 34us/step - loss: 0.2031 - acc: 0.7954 - val_loss: 0.1954 - val_acc

In [6]:
weights = autoencoder.get_weights()
for idx, weight in enumerate(weights):
    print("LAYER {}".format(idx))
    print(weight)

LAYER 0
[[-0.00886185  0.05947391  0.0362878  ...  0.0643462  -0.0302072
   0.06987628]
 [-0.05570446  0.05679414 -0.00288785 ... -0.02871788  0.01562922
  -0.06326306]
 [-0.04213982  0.00370272 -0.01871976 ...  0.01576682  0.01789319
  -0.06476408]
 ...
 [-0.06354181 -0.06226511  0.01426818 ...  0.04273959  0.07934282
   0.07521937]
 [-0.02018193 -0.00297017  0.04712795 ... -0.01389293 -0.04841049
   0.00758218]
 [-0.07920329 -0.00101845  0.0739345  ... -0.07975397 -0.03917845
  -0.01054192]]
LAYER 1
[ 1.63487345e-01  6.87050596e-02  3.19820940e-01 -1.67464048e-01
  1.07701115e-01  2.27858588e-01  1.94374949e-01  9.55232605e-02
  1.64251581e-01 -2.09099539e-02  1.85118601e-01  1.85812250e-01
 -1.25695691e-02  5.39480269e-01  3.06804515e-02  4.04111862e-01
  9.15872082e-02  6.76475316e-02 -2.83332579e-02  8.15361142e-02
  2.71564811e-01 -1.06186643e-02  2.79324315e-02  9.17478949e-02
 -3.39823291e-02  9.61285308e-02  7.00145811e-02  1.04808070e-01
  7.92038590e-02 -1.29371345e-01  1.24

In [0]:
from keras.models import Sequential

num_classes = 10
img_size = len(x_train[0])
encoding_dim = len(x_train_encoded[0])

mnist_encoded_classifier = Sequential()
mnist_encoded_classifier.add(Dense(units=64, activation='selu', input_shape=(encoding_dim,)))
mnist_encoded_classifier.add(Dense(units=32, activation='selu', input_shape=(encoding_dim,)))
mnist_encoded_classifier.add(Dense(units=num_classes, activation='softmax'))
mnist_encoded_classifier.compile(optimizer="sgd", loss='categorical_crossentropy', metrics=['accuracy'])
mnist_encoded_classifier.summary()

mnist_encoded_classifier.fit(x_train_encoded, y_train,
                            epochs=50,
                            batch_size=256,
                            validation_data=(x_test_encoded, y_test))



Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_35 (Dense)             (None, 64)                2112      
_________________________________________________________________
dense_36 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_37 (Dense)             (None, 10)                330       
Total params: 4,522
Trainable params: 4,522
Non-trainable params: 0
_________________________________________________________________
Train on 60000 samples, validate on 10000 samples
Epoch 1/50
60000/60000 [==============================] - 2s 26us/step - loss: 0.8474 - acc: 0.7455 - val_loss: 0.4517 - val_acc: 0.8600
Epoch 2/50
60000/60000 [==============================] - 1s 17us/step - loss: 0.4135 - acc: 0.8734 - val_loss: 0.3850 - val_acc: 0.8764
Epoch 3/50
60000/60000 [==============================] -